In [1]:
# Initially with openmm
#!/usr/bin/env python3
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout
import os
import sys
import argparse
import nglview as nv
import pytraj as pt
import pdbfixer

_ColormakerRegistry()

Easiest way to fix downloaded PDB files to be processed by OpenMM is to run pdbfixer from the command line and use the HTML interface that shows up.
Easy as pie!
Ref: https://htmlpreview.github.io/?https://raw.github.com/pandegroup/pdbfixer/master/Manual.html


In [34]:
#pdb= PDBFile('data/6lvn.pdb')
pdb = PDBFile('data/6lvn_fixed.pdb')
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
modeller = Modeller(pdb.topology, pdb.positions)
modeller.addHydrogens(forcefield)
print(modeller.topology)

<Topology; 8 chains, 166 residues, 2342 atoms, 2316 bonds>


In [8]:
platform = Platform.findPlatform()

TypeError: findPlatform() missing 1 required positional argument: 'kernelNames'

In [18]:
# from https://www.ncbi.nlm.nih.gov/protein/QHD43417/
pdb= PDBFile('data/QHD43417.pdb')
forcefield = ForceField('amber03.xml', 'amber03_obc.xml')
modeller = Modeller(pdb.topology, pdb.positions)
#modeller.addHydrogens(forcefield)
#print(modeller.topology)

In [10]:
# from https://www.rcsb.org/structure/1vii
pdb2 = PDBFile('data/1vii.pdb')

forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
#forcefield = ForceField('amber03.xml', 'amber03_obc.xml')
modeller = Modeller(pdb2.topology, pdb2.positions)
modeller.addHydrogens(forcefield)
print(modeller.topology)

<Topology; 1 chains, 36 residues, 596 atoms, 602 bonds>


In [28]:
# from https://www.rcsb.org/structure/1vii
pdb2 = PDBFile('data/2prf.pdb')

forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
#forcefield = ForceField('amber03.xml', 'amber03_obc.xml')
modeller = Modeller(pdb2.topology, pdb2.positions)
modeller.addHydrogens(forcefield)
print(modeller.topology)

<Topology; 1 chains, 125 residues, 1826 atoms, 1842 bonds>


In [ ]:
system = forcefield.createSystem(modeller.topology,
  implicitSolvent=OBC2,   # matches https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2980750/#bib39
  nonbondedMethod=NoCutoff, nonbondedCutoff=1*nanometer,
  constraints=HBonds)
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 2*femtoseconds)
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)
simulation.minimizeEnergy()

steps = 100000
steps_write = max(1, steps//1000)
print("writing every %d steps" % steps_write)
simulation.reporters.append(PDBReporter('output.pdb', 1000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,potentialEnergy=True, temperature=True))
simulation.step(steps)

writing every 100 steps
#"Step","Potential Energy (kJ/mole)","Temperature (K)"
1000,-9644.40020582318,270.0506294986803
2000,-8680.520795637349,296.1787257108011
3000,-9097.036088179564,304.7337254578176
4000,-8954.231014715442,301.8824781533834
5000,-9346.255330631637,297.89822392555806
6000,-9357.570242687116,296.46932742231485
7000,-9675.009728839988,301.9894399042805
8000,-9703.878662353678,302.96458276900637
9000,-9616.550062142593,305.0325355038976
10000,-9884.111369013935,305.2985066260844
11000,-9566.876458314979,303.2917890009863
12000,-9662.805238462446,295.6221499845513
13000,-9486.264810641647,298.9411822912113
14000,-9865.326300558943,300.5836706081506
15000,-9594.967312642337,305.2221492046354
16000,-9649.80303714579,299.3665647701112
17000,-9940.932724086933,301.43139805947516


In [5]:
traj = pt.load('/tmp/output.pdb')
view = nv.show_pytraj(traj)
view

NGLWidget(max_frame=999)